<a href="https://colab.research.google.com/github/jsvillalbat/image-to-image-similarity/blob/main/scrape_image_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Scrape Images
Cuaderno para hacer scrapping de imagenes de Google con el fin de obtener imagenes para entrenar un modelo de image retrieval sobre herramientas y productos de construcción para TUL.

Basado en este repositorio publicod e Github: https://github.com/ankonzoid/artificio/blob/master/scrape_google_images/scrape_google_images.py

In [ ]:
# import the libraries
import os, json, time
import requests
import urllib.request as urllibreq
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
absolute_path = "/content/drive/MyDrive/Colab Notebooks/Image Retrieval"
train_dataset_path = absolute_path + "/train_1"
test_dataset_path = absolute_path + "/test_1"

Función que realiza el scrapping de imagenes de Google, recibe como párametros el query, u objeto a hacer scrapping, el número de imagenes a descargar, el directorio donde guardar las imagenes y el número de segundos a esperar entre la descarga de las imagenes

In [ ]:
def scrape_google_images(query, k, outDir, dt_stall):
    print("Scraping top k={} Google images for query: '{}'".format(k, query))

    # Create output directory
    if not os.path.exists(outDir):
        os.mkdir(outDir)
    # Construct image url to scrape from
    url = "https://www.google.co.in/search?q=" + \
          '+'.join(query.split()) + "&source=lnms&tbm=isch"
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}

    # Download html + parse it to get all image urls
    html = urllibreq.urlopen(urllibreq.Request(url, headers=header))
    soup = BeautifulSoup(html,'html.parser')
    urls = [ x.find('img')['src']
            for x in soup.find_all("div", {"class": "kCmkOe"})]
    print("Parsed {} google image urls!".format(len(urls)))

    # Download top-k images
    for i, url in enumerate(urls[:min(k, len(urls))]):
        imgFile = os.path.join(outDir, query + "-" + str(i+1) + ".jpg")
        print("[{}/{}] Downloading image {} to '{}'...".format(i+1, k, url, imgFile))
        try:
            with open(imgFile, 'wb') as handle:
                response = requests.get(url, stream=True)
                if not response.ok:
                    print(response)
                for block in response.iter_content(1024):
                    if not block:
                        break
                    handle.write(block)
            time.sleep(dt_stall)  # force pause
        except:
            raise Exception("  -> Failed downloading: " + url)

Correr función para descargar imagenes

In [ ]:
query = "masking tape"  # google images query text
k = 5  # top-k images will be scraped
dt_stall = 2  # number of seconds to stall between image scrapes
scrape_google_images(query=query, k=k, outDir=train_dataset_path, dt_stall=dt_stall)

Scraping top k=5 Google images for query: 'masking tape'
Parsed 20 google image urls!
[1/5] Downloading image https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDkM7NqHDNlBIT8PBw-yiSdW3OPCuQd9brEu7eNnIOgMkB2wpNcvP8UyWi9g&s to '/content/drive/MyDrive/Colab Notebooks/Image Retrieval/train_1/masking tape-1.jpg'...
[2/5] Downloading image https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTAeTB6PJs-bW3K2Fej86E7QQmihTV4Zl0ya9j55ce6zqJe2ar72NDrBzDvug&s to '/content/drive/MyDrive/Colab Notebooks/Image Retrieval/train_1/masking tape-2.jpg'...
[3/5] Downloading image https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQc1u4bFXSad55jooJ7GO_i1dGtDujYmZBbVmILbae_ga9GlMbE6lZ19kX05Q&s to '/content/drive/MyDrive/Colab Notebooks/Image Retrieval/train_1/masking tape-3.jpg'...
[4/5] Downloading image https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS-raAtHZHHY4yv69NOQ2nk3MwZVvgraZbrf0dTojoG24zEdMEeLf83eOyOanU&s to '/content/drive/MyDrive/Colab Notebooks/Image Retrieval/train_1/masking tape